In [16]:
import json
import requests
import pandas as pd
from tqdm import tqdm
from typing import NamedTuple
from datetime import datetime

In [17]:
class Currency_info(NamedTuple):
    names: tuple
    periods: tuple

In [18]:
def Get_maturity_date_from_option_name(option_name: str):
    date_str = option_name.split("-")[1]
    maturity_date = datetime.strptime(date_str, '%d%b%y')
    return maturity_date 

In [19]:
def Get_days_till_maturity(option_name: str):
    date_str = option_name.split("-")[1]
    maturity_date = datetime.strptime(date_str, '%d%b%y')    
    return ((maturity_date - datetime.today()).days)

In [20]:
def Get_options_information(currency_name: str) -> Currency_info:
    """
    Get instrument_name and settlement_period for all options of selected currency from deribit (public)
    """
    request = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + currency_name + "&kind=option")
    request_text = json.loads(request.text)
    currency_info = Currency_info(tuple(pd.json_normalize(request_text['result'])['instrument_name']),
                                  tuple(pd.json_normalize(request_text['result'])['settlement_period']))

    return currency_info


def Get_options_data(currency_name: str) -> pd.DataFrame:
    """
    Get currency data for instrument names
    """
    currency_info = Get_options_information(currency_name)
    progress_bar = tqdm(total = len(currency_info.names))
    currency_data = []
    for i in range(len(currency_info.names)):
        request = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + currency_info.names[i])
        request_text = json.loads(request.text)
        add_data = pd.json_normalize(request_text['result'])
        add_data['settlement_period'] = currency_info.periods[i]
        add_data['option_name'] = currency_info.names[i]
        add_data['strike'] = int(currency_info.names[i].split("-")[2])
        add_data['maturity_date'] = Get_maturity_date_from_option_name(currency_info.names[i])
        add_data['days_till_maturity'] = Get_days_till_maturity(currency_info.names[i])        
        currency_data.append(add_data)
        progress_bar.update(1)
    currency_dataframe = pd.concat(currency_data)
    """currency_dataframe = currency_dataframe[['underlying_price', 'timestamp', 'settlement_price',
                        'open_interest', 'min_price', 'max_price', 'mark_price', 'mark_iv',
                        'last_price', 'interest_rate', 'instrument_name', 'index_price', 'bids',
                        'bid_iv', 'best_bid_price', 'best_bid_amount','best_ask_price', 
                        'best_ask_amount', 'asks', 'ask_iv', 'stats.volume','stats.price_change',
                        'stats.low', 'stats.high','settlement_period','strike', 'option_name']].copy"""
    currency_dataframe = currency_dataframe[['instrument_name', 'underlying_price', 'timestamp', 'mark_price', 'mark_iv',
                         'best_bid_price', 'best_ask_price', 'strike', 'option_name', 'maturity_date', 'days_till_maturity']].copy()                    
    progress_bar.close()
    return currency_dataframe

In [21]:

BTC_options_data = Get_options_data('BTC')
print(BTC_options_data)

100%|██████████| 314/314 [01:04<00:00,  4.89it/s]

        instrument_name  underlying_price      timestamp  mark_price  mark_iv  \
0   BTC-11NOV22-15000-C          17846.54  1668116894631      0.1597   210.00   
0   BTC-11NOV22-15000-P          17845.94  1668116895638      0.0002   210.00   
0   BTC-11NOV22-16000-C          17845.96  1668116895642      0.1053   210.00   
0   BTC-11NOV22-16000-P          17845.96  1668116895643      0.0019   210.00   
0   BTC-11NOV22-17000-C          17845.94  1668116895641      0.0563   195.09   
..                  ...               ...            ...         ...      ...   
0   BTC-29SEP23-30000-P          17731.54  1668116958074      0.7415    53.74   
0   BTC-29SEP23-40000-C          17731.54  1668116958074      0.0168    53.74   
0   BTC-29SEP23-40000-P          17731.54  1668116958075      1.2727    53.74   
0   BTC-29SEP23-50000-C          17731.54  1668116958074      0.0061    53.74   
0   BTC-29SEP23-50000-P          17731.64  1668116959080      1.8259    53.74   

    best_bid_price  best_as

In [22]:
print(BTC_options_data)

        instrument_name  underlying_price      timestamp  mark_price  mark_iv  \
0   BTC-11NOV22-15000-C          17846.54  1668116894631      0.1597   210.00   
0   BTC-11NOV22-15000-P          17845.94  1668116895638      0.0002   210.00   
0   BTC-11NOV22-16000-C          17845.96  1668116895642      0.1053   210.00   
0   BTC-11NOV22-16000-P          17845.96  1668116895643      0.0019   210.00   
0   BTC-11NOV22-17000-C          17845.94  1668116895641      0.0563   195.09   
..                  ...               ...            ...         ...      ...   
0   BTC-29SEP23-30000-P          17731.54  1668116958074      0.7415    53.74   
0   BTC-29SEP23-40000-C          17731.54  1668116958074      0.0168    53.74   
0   BTC-29SEP23-40000-P          17731.54  1668116958075      1.2727    53.74   
0   BTC-29SEP23-50000-C          17731.54  1668116958074      0.0061    53.74   
0   BTC-29SEP23-50000-P          17731.64  1668116959080      1.8259    53.74   

    best_bid_price  best_as

In [23]:
print(Get_options_information("BTC"))

Currency_info(names=('BTC-11NOV22-15000-C', 'BTC-11NOV22-15000-P', 'BTC-11NOV22-16000-C', 'BTC-11NOV22-16000-P', 'BTC-11NOV22-17000-C', 'BTC-11NOV22-17000-P', 'BTC-11NOV22-18000-C', 'BTC-11NOV22-18000-P', 'BTC-11NOV22-19000-C', 'BTC-11NOV22-19000-P', 'BTC-11NOV22-20000-C', 'BTC-11NOV22-20000-P', 'BTC-11NOV22-21000-C', 'BTC-11NOV22-21000-P', 'BTC-11NOV22-22000-C', 'BTC-11NOV22-22000-P', 'BTC-11NOV22-25000-C', 'BTC-11NOV22-25000-P', 'BTC-12NOV22-13000-C', 'BTC-12NOV22-13000-P', 'BTC-12NOV22-14000-C', 'BTC-12NOV22-14000-P', 'BTC-12NOV22-14500-C', 'BTC-12NOV22-14500-P', 'BTC-12NOV22-15000-C', 'BTC-12NOV22-15000-P', 'BTC-12NOV22-15500-C', 'BTC-12NOV22-15500-P', 'BTC-12NOV22-16000-C', 'BTC-12NOV22-16000-P', 'BTC-12NOV22-16250-C', 'BTC-12NOV22-16250-P', 'BTC-12NOV22-16500-C', 'BTC-12NOV22-16500-P', 'BTC-12NOV22-16750-C', 'BTC-12NOV22-16750-P', 'BTC-12NOV22-17000-C', 'BTC-12NOV22-17000-P', 'BTC-12NOV22-17250-C', 'BTC-12NOV22-17250-P', 'BTC-12NOV22-17500-C', 'BTC-12NOV22-17500-P', 'BTC-12NOV22-

In [24]:
print(Get_days_till_maturity("BTC-29SEP23-30000-P"))

321
